<a href="https://colab.research.google.com/github/sonmh79/simple_VRP_GA/blob/main/VRP_robot_delivery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
d = [[0, 60, 80, 110, 120, 160, 140, 150, 165, 85, 80],
 [60, 0, 20, 50, 60, 100, 120, 130, 145, 65, 60],
 [80, 20, 0, 30, 40, 80, 100, 150, 165, 85, 80],
 [110, 50, 30, 0, 10, 50, 70, 80, 95, 115, 110],
 [120, 60, 40, 10, 0, 40, 60, 110, 125, 125, 120],
 [160, 100, 80, 50, 40, 0, 20, 70, 85, 165, 160],
 [140, 120, 100, 70, 60, 20, 0, 50, 65,145, 140],
 [150, 130, 150, 80, 110, 70, 50, 0, 15, 155, 150],
 [165, 145, 165, 95, 125, 85, 65, 15, 0, 170, 165],
 [85, 65, 85, 115, 125, 165, 145, 155, 170, 0, 5],
 [80, 60, 80, 110, 120, 160, 140, 150, 165, 5, 0]]

In [3]:
d_data = pd.DataFrame(d)
d_data

,0,1,2,3,4,5,6,7,8,9,10
0,0,60,80,110,120,160,140,150,165,85,80
1,60,0,20,50,60,100,120,130,145,65,60
2,80,20,0,30,40,80,100,150,165,85,80
3,110,50,30,0,10,50,70,80,95,115,110
4,120,60,40,10,0,40,60,110,125,125,120
5,160,100,80,50,40,0,20,70,85,165,160
6,140,120,100,70,60,20,0,50,65,145,140
7,150,130,150,80,110,70,50,0,15,155,150
8,165,145,165,95,125,85,65,15,0,170,165
9,85,65,85,115,125,165,145,155,170,0,5


In [4]:
capa = {1: 1, 2: 2, 3: 1, 4: 3, 5: 3, 6: 1, 7: 3, 8: 2, 9: 2, 10: 3}

In [5]:
data = [[0,0,0]]
for i in range(1,11):
  data.append([0,0,capa[i]])
data

[[0, 0, 0],
 [0, 0, 1],
 [0, 0, 2],
 [0, 0, 1],
 [0, 0, 3],
 [0, 0, 3],
 [0, 0, 1],
 [0, 0, 3],
 [0, 0, 2],
 [0, 0, 2],
 [0, 0, 3]]

In [6]:
df = pd.DataFrame(data)
df.columns = ["x","y","c"]
df

,x,y,c
0,0,0,0
1,0,0,1
2,0,0,2
3,0,0,1
4,0,0,3
5,0,0,3
6,0,0,1
7,0,0,3
8,0,0,2
9,0,0,2


In [7]:
import random
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


class VRP:
  def __init__(self,dataset,d_data = d_data,capacity=100,cnum=100,mutation_prob=0.2,ev_times=100):
    self.n = len(dataset)
    self.c = capacity
    self.root = [dataset.loc[0]["x"],dataset.loc[0]["y"]]
    self.data = dataset
    self.d_data = d_data
    self.cnum = cnum
    self.cnt = 0
    self.mutation_prob = mutation_prob
    self.ev_times = ev_times
    self.dist = 0
    self.min_route = []
    self.info = []

  #make many chromos...
  def _chromo(self,t=1):
    res = []
    for _ in range(t):
     res.append(random.sample(range(1,self.n),self.n-1)) 
    return res


  def _calc_route(self,route):
    res = 0
    for i in range(len(route)-1):
      res += self.d_data.loc[route[i]][route[i+1]]
    return res
  
  def _make_route(self,chromo):
      route = []
      border = self.routing(chromo)
      for i in range(len(border)):
        if i == 0:
          route = route + [0] + chromo[:border[i]+1] + [0]
        elif i == len(border)-1:
          route = route + [0] + chromo[border[i-1]+1:border[i]+1] + [0]
          route = route + [0] + chromo[border[i]+1:] + [0]
        else:
          route = route + [0] + chromo[border[i-1]+1:border[i]+1] + [0]
      return route

  #get distance of each route
  def _getdist(self, chromos):
    dist_list = []

    for chromo in chromos:
      route = self._make_route(chromo)
      dist_list.append(self._calc_route(route))

    fit = list(zip(dist_list,chromos))
    fit.sort(key = lambda x:x[0])
    return fit

  def _rand(self,x,y):
    return int(random.uniform(x,y))

  def _crossover(self,p1,p2,hard_mode=False):
    swap_point = self._rand(1,len(p1))
    c1,c2 = [],[]

    i = 0
    while i < swap_point:
      c1.append(p1[i])
      c2.append(p2[i])
      i+=1

    for e in p2:
      if e not in c1:
        c1.append(e)

    for e in p1:
      if e not in c2:
        c2.append(e)
    
    #mutation
    if random.uniform(0,1) <= self.mutation_prob and len(c1)>2 and not hard_mode:
      e1,e2 = random.sample(range(1,len(c1)),2)
      #target will select mutation child
      if e1 > e2:
        target = e1
      else:
        target = e2
      
      if target%2 == 0:
        c1[e1],c1[e2] = c1[e2],c1[e1]
      else:  
        c2[e1],c2[e2] = c2[e2],c2[e1]

    if hard_mode:
      e1,e2 = random.sample(range(0,len(c1)-1),2)
      c1[e1],c1[e1+1] = c1[e1+1],c1[e1]
      c2[e2],c2[e2+1] = c2[e2+1],c2[e2]

    next_gen = self._getdist([c1,c2,p1,p2])
    #print("next_gen: ",next_gen)
    c1,c2 = next_gen[0][1],next_gen[1][1]
    #print("c1,c2: ",c1,c2)
    return [c1,c2]

  
  def _select_parent(self,parents):
    fitness = [1/route[0] for route in parents] 
    sum = 0

    for f in fitness:
      sum += f
    p_index = set()

    while len(p_index)<2:
      fit = 0
      target = random.uniform(0,sum)
      for i in range(len(fitness)):
        fit += fitness[i]
        if fit > target:
          p_index.add(i)
          break  
    return list(p_index)

  
  def _make_child(self,dist_routes):
    self.cnt += 1
    parents = dist_routes.copy()
    routes = [route[1] for route in parents]
    child = []

    # select top 100 chromos
    while parents and len(child)<self.cnum:
      selected_parents = self._select_parent(parents)
      p1,p2 = routes.pop(selected_parents[0]),routes.pop(selected_parents[1]-1)
      parents.pop(selected_parents[0]),parents.pop(selected_parents[1]-1)
      c1,c2 = self._crossover(p1,p2)
      child += c1,c2

    return child

  def evolution(self,chromo):
    
    dist_routes = self._getdist(chromo) #(dist,routes)
    child = self._make_child(dist_routes)
    return child

  def routing(self,chromo):
    cap = 0
    border_index = []
    for i in range(len(chromo)):
      cap += self.data.loc[chromo[i]]["c"]
      if i != 0 and cap > self.c:
        cap = self.data.loc[chromo[i]]["c"]
        border_index.append(i-1)
    return border_index

  def ga(self): 
    chromos = self._chromo(100)    
    gen = self.evolution(chromos)
    self.info = self._getdist(gen)
    print("first genenration minimun distance: ",(self.info[0][0],self.info[0][1]))
    for i in range(self.ev_times):
      gen = self.evolution(gen)
      self.info = self._getdist(gen)
      t = self.cnt
      if t % 10 == 0:
        print("{}/{}th evolution min distance: ".format(t,self.ev_times),(self.info[0][0],self.info[0][1]))
      
      # to avoid local minimum
      if t == 30:
        self.mutation_prob = t/100
        print("==========Mutation prob upgrade!==========: ",self.mutation_prob)
      elif t == 50:
        self.mutation_prob = t/100
        print("==========Mutation prob upgrade!==========: ",self.mutation_prob)
      elif t == 70:
        self.mutation_prob = t/100
        print("==========Mutation prob upgrade!==========: ",self.mutation_prob)
      
    self.dist,self.min_route = self.info[0][0],self.info[0][1]   
    print("----------------------------------------Finish----------------------------------------")
    print("Total travel time / Min route: ",self.dist," / ",self.min_route)
    route = self._make_route(self.min_route)
    z = 0
    t = 0
    p = ""
    for r in route:
      if r == 0 and z != 1:
        z += 1
        p += str(r)
        p += "->"
      elif r == 0 and z == 1:
        t += 1
        z = 0
        p += str(r)
        print("R{}: ".format(t)+p)
        p =""
      else:
        p += str(r)
        p += "->"
    
#if __name__ == "__main__":
  


In [8]:
import time
v = VRP(df,d_data,capacity=3,ev_times=10)
s= time.time()
v.ga()
e = time.time()
print("Total time: ",e-s,"(s)")

first genenration minimun distance:  (1820, [10, 1, 9, 4, 5, 3, 2, 7, 6, 8])
10/10th evolution min distance:  (1820, [10, 3, 2, 7, 4, 6, 8, 9, 1, 5])
----------------------------------------Finish----------------------------------------
Total travel time / Min route:  1820  /  [10, 1, 9, 5, 4, 8, 6, 3, 2, 7]
R1: 0->10->0
R2: 0->1->9->0
R3: 0->5->0
R4: 0->4->0
R5: 0->8->6->0
R6: 0->3->2->0
R7: 0->7->0
Total time:  17.427202701568604 (s)


In [9]:
v.routing(v.min_route)

[0, 2, 3, 4, 6, 8]